### 3-3．모델작성
#### 데이터의 리드와 확인

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Pandas의 로드
import pandas as pd

# 파일 리드
bank_df_new = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/머신러닝 데이터 전처리 입문_로드북/chap3-4/data/bank-prep.csv', sep=',')

# 선두에서 5행까지 표시
bank_df_new.head()

,age,default,balance,housing,loan,day,duration,campaign,pdays,previous,...,dec,feb,jan,jul,jun,mar,may,nov,oct,sep
0,58,0,2143,1,0,5,261,1,-1,0,...,0,0,0,0,0,0,1,0,0,0
1,36,0,265,1,1,5,348,1,-1,0,...,0,0,0,0,0,0,1,0,0,0
2,25,0,-7,1,0,5,365,1,-1,0,...,0,0,0,0,0,0,1,0,0,0
3,53,0,-3,0,0,5,1666,1,-1,0,...,0,0,0,0,0,0,1,0,0,0
4,24,0,-103,1,1,5,145,1,-1,0,...,0,0,0,0,0,0,1,0,0,0


#### 불균형데이터의 균형화

In [3]:
!pip show imbalanced-learn

Name: imbalanced-learn
Version: 0.10.1
Summary: Toolbox for imbalanced dataset in machine learning.
Home-page: https://github.com/scikit-learn-contrib/imbalanced-learn
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.9/dist-packages
Requires: joblib, numpy, scikit-learn, scipy, threadpoolctl
Required-by: 


In [4]:
!pip list

Package                       Version
----------------------------- --------------------
absl-py                       1.4.0
alabaster                     0.7.13
albumentations                1.2.1
altair                        4.2.2
anyio                         3.6.2
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arviz                         0.15.1
astropy                       5.2.2
astunparse                    1.6.3
attrs                         22.2.0
audioread                     3.0.0
autograd                      1.5
Babel                         2.12.1
backcall                      0.2.0
beautifulsoup4                4.11.2
bleach                        6.0.0
blis                          0.7.9
blosc2                        2.0.0
bokeh                         2.4.3
branca                        0.6.0
CacheControl                  0.12.11
cached-property               1.5.2
cachetools                    5.3.0
cata

In [5]:
!pip install imbalanced-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
# Numpy와 RandomUnderSampler의 로드
import numpy as np
from imblearn.under_sampling import RandomUnderSampler

# 데이터 세트를 설명변수와 목적변수로 분할
X = np.array(bank_df_new.drop('y', axis=1)) # axis=1 :열 제거  # 판다스 시리즈 -> 넘파이 어레이로 바꿈
Y = np.array(bank_df_new['y'])  # 판다스 시리즈 -> 넘파이 어레이로 바꿈

# y가 1과 0인 데이터의 건수를 카운트
print(np.sum(Y == 1), np.sum(Y == 0))
 
# y가 1인 데이터의 건수에 맞춰서 다운사이징
sampler = RandomUnderSampler(random_state=42)
X, Y = sampler.fit_resample(X, Y)

# y가 1과 0인 데이터 건수를 카운트
print(np.sum(Y == 1), np.sum(Y == 0))

820 6113
820 820


In [7]:
X # 어레이로 할때 숫자처리 할때 용이

array([[  35,    0,   81, ...,    0,    0,    0],
       [  22,    0,  395, ...,    0,    0,    0],
       [  59,    0,   93, ...,    0,    0,    0],
       ...,
       [  38,    0, 2543, ...,    1,    0,    0],
       [  28,    0,  159, ...,    1,    0,    0],
       [  25,    0,  505, ...,    1,    0,    0]])

1640

In [8]:
Y # 어레이로 할때 숫자처리 할때 용이

array([0, 0, 0, ..., 1, 1, 1])

실제로 예측하고자 하는 데이터 수가 많지 않는때가 있다 그것은 고장난 로그는 축적되지 않아서 이것은 오버샘플링을 해야 한다. 

In [ ]:
# 오버샘플링

import numpy as np
from imblearn.over_sampling import RandomOverSampler

X = np.array(bank_df_new.drop("y", axis =1)
Y = np.array(bank_df_new['y'])

print(np.sum(Y == 1), np.sum(X == 1))

sampler = RandomOverSampler(random_state = 42)
X, Y = sampler.fit_resampler(X, Y)
print(np.sum(Y == 1, np.sum(Y == 0))

SMOTE : 복사하지 않고 보완해서 데이터를 증가시키는 방법이 좀 더 일반적이다. 

이것도 해보다. 숙제.
아이디어는 K-NN 기법 즉 K-최근접 이웃 알고리즘이다.

#### 결정트리의 작성과 검증

In [18]:
# Scikit-learn의 로드
from sklearn.model_selection import KFold # 모델 검증
from sklearn import tree # 모델 적용
from sklearn.metrics import accuracy_score # 모델 평가

# K-Fold 교차검증
kf = KFold(n_splits=10, shuffle=True)

# 모델의 정밀도를 보존할 준비
scores = []

# 데이터를 셔플하고, 훈련데이터와 테스트데이트로 분할
for train_id, test_id in kf.split(X):
    
    # 훈련 데이터를 사용하여 모델을 작성
    x = X[train_id]
    y = Y[train_id]
    clf = tree.DecisionTreeClassifier()
    clf.fit(x,y)

    # 테스트 데이터에 모델을 적용
    pred_y = clf.predict(X[test_id])
    
    # 모델의 정밀도를 계산해서 보존
    score = accuracy_score(Y[test_id], pred_y)
    scores.append(score)

# 모델의 평균정밀도, 표준편차를 확인
scores = np.array(scores)
print(scores.mean(), scores.std())

0.7658536585365854 0.02783588344027641


In [29]:
X.shape

(1640, 31)

In [30]:
Y.shape

(1640,)

In [31]:
# Scikit-learn의 로드
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

# 재현률, 적합률의 확인
#print(confusion_matrix(Y[test_id], pred_y))
print(recall_score(Y[test_id], pred_y))  # 재현율 확인
print(precision_score(Y[test_id], pred_y))  # 적합율 확인

0.6805555555555556
0.6901408450704225


#### 모델의 정밀도 향상

In [36]:
# 모델작성용 패러메터를 확인
help(clf)

# criterion 분할지표 지니계수 
# 트리의 최대 깊이 max_depth
# min_samples_leaf 1이 지정된다.


Help on DecisionTreeClassifier in module sklearn.tree._classes object:

class DecisionTreeClassifier(sklearn.base.ClassifierMixin, BaseDecisionTree)
 |  DecisionTreeClassifier(*, criterion='gini', splitter='best', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, class_weight=None, ccp_alpha=0.0)
 |  
 |  A decision tree classifier.
 |  
 |  Read more in the :ref:`User Guide <tree>`.
 |  
 |  Parameters
 |  ----------
 |  criterion : {"gini", "entropy", "log_loss"}, default="gini"
 |      The function to measure the quality of a split. Supported criteria are
 |      "gini" for the Gini impurity and "log_loss" and "entropy" both for the
 |      Shannon information gain, see :ref:`tree_mathematical_formulation`.
 |  
 |  splitter : {"best", "random"}, default="best"
 |      The strategy used to choose the split at each node. Supported
 |      strategies are "best" to

In [38]:
# Scikit-learn을 로드
from sklearn.model_selection import GridSearchCV

# 패러메터 범위설정
params = {     # 하이퍼파라미터 
    'criterion': ['entropy'],    
    'max_depth': [2, 4, 6, 8, 10],
    'min_samples_leaf': [10, 20, 30, 40, 50],
}

# 그리드 서치의 실행조건을 설정 
clf_gs = GridSearchCV(tree.DecisionTreeClassifier(), params,  # 첫번쨰 인자에는 알고리즘, 즉 분류의 결정 트리를, 두번째 인자에는 탐색하는 파라미터 세트를, 세번째 인자에는 검증방법, 즉 K-폴드 교차검증, 네번째인자는 평가방법, 즉 모델의 성능을 지정한다.
                      cv=KFold(n_splits=10, shuffle=True), scoring='accuracy') # 이거는 파라미터

# 그리드 서치 실행(=훈련)
clf_gs.fit(X, Y) 

GridSearchCV(cv=KFold(n_splits=10, random_state=None, shuffle=True),
             estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['entropy'],
                         'max_depth': [2, 4, 6, 8, 10],
                         'min_samples_leaf': [10, 20, 30, 40, 50]},
             scoring='accuracy')

In [44]:
# 가장 높은 정밀도와 패터메터의 조합을 표시
print(clf_gs.best_score_)
print(clf_gs.best_params_)

0.7987804878048781
{'criterion': 'entropy', 'max_depth': 6, 'min_samples_leaf': 20}


In [53]:
# 최적의 패터메터를 사용한 모델을 작성
clf_best = tree.DecisionTreeClassifier(criterion='entropy', max_depth=6, min_samples_leaf=20)
clf_best.fit(X, Y)

# 영향이 높은 변수를 확인
pd.DataFrame(clf_best.feature_importances_ * 100).T

# 영향력이 낮은 변수는 그대로 사용하고 있기 때문에 영향력이 낮게 나온 것일수도 잇다. 좀 더 레벨이 높은 전처리르 사용해 기존변수로 부터 신규변수를 작성하면 신규변수는 영향력이 높은 것이될 가능성도 잇다. 

# 해당 변수가 어떤것이 매칭은 어떻게 확인할까?

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,0.555041,0.0,3.06564,10.278582,1.832438,2.880909,59.030607,1.041402,7.230604,4.523754,...,0.0,0.0,0.0,0.0,0.0,0.0,2.296423,0.0,0.0,0.0


In [54]:
X

array([[  35,    0,   81, ...,    0,    0,    0],
       [  22,    0,  395, ...,    0,    0,    0],
       [  59,    0,   93, ...,    0,    0,    0],
       ...,
       [  38,    0, 2543, ...,    1,    0,    0],
       [  28,    0,  159, ...,    1,    0,    0],
       [  25,    0,  505, ...,    1,    0,    0]])